In [1]:
using TSML, TSMLextra, Plots

In [2]:
using TSML: TSClassifier
Random.seed!(12)

trdirname = joinpath(dirname(pathof(TSML)),"../data/realdatatsclassification/training")
tstdirname = joinpath(dirname(pathof(TSML)),"../data/realdatatsclassification/testing")
modeldirname = joinpath(dirname(pathof(TSML)),"../data/realdatatsclassification/model")

tscl = TSClassifier(Dict(:trdirectory=>trdirname,
           :tstdirectory=>tstdirname,
           :modeldirectory=>modeldirname,
           :feature_range => 6:20,
           :num_trees=>10)
)

fit!(tscl)
dfresults = transform!(tscl)

UndefVarError: UndefVarError: TSClassifier not defined

    Status `~/julia/TSML.jl/docs/Project.toml`
  [336ed68f] CSV v0.5.5
  [a93c6f00] DataFrames v0.18.2
  [e30172f5] Documenter v0.22.4
  [35a29f4d] DocumenterTools v0.1.0
  [23fbe1c1] Latexify v0.8.2
  [198dc43e] TSML v2.3.0
  [ade2ca70] Dates 
  [9a3f8284] Random 
